# Primer Proyecto de DAA
*Equipo: pss, it's broken*

Integrantes:
+ Niley González Ferrales
+ Gabriel Hernández Rodríguez

Problema: **Tito el Tramposo**

## Texto del problema
El examen tiene $n$ preguntas, ordenadas en la hoja. Elena y Hansel pueden no ser capaces de responder cada uno el examen entero, pero todas las preguntas que responden, están correctas. Se conoce cuáles preguntas respondió cada uno y se reciben como dos listas ordenadas de enteros entre $1$ y $n$. Tito tiene $p$ oportunidades para mirar hacia la izquierda (hoja de Hansel) o hacia la derecha (Hoja de Elena) y su agilidad mental le alcanza para ver las respuestas de $k$ preguntas consecutivas (en cualquier posición) cada vez que echa una mirada a un examen. Ayude a Tito a saber la cantidad máxima de preguntas que puede responder con su (tramposa) estrategia.

## Problema concreto
Dado un entero positivo $n$, 2 listas ordenadas ($A$ y $B$) de enteros entre $1$ y $n$, y $2$ enteros positivos: $k$ y $p$. Devuelva la mayor cantidad de números distintos que se pueden ver entre ambas listas mirando solo $p$ veces. Se pueden ver, en cada mirada, hasta $k$ números. O sea, dada una posición $i$, una lista $L$ y un valor $k'\leq k$ se consideran vistos todos los números $x$ tal que $i\leq x \leq i+k'-1 \land x\in L$.

Se asume que:
+ El valor de $k$ es menor igual que $n$. Si fuera mayor asumir que es igual a $n$ y la solución es la misma.
+ El valor de $p$ es menor igual que $2n$. Si fuera mayor se repiten obligatoriamente posiciones así que se puede asumir que $p$ es igual a $2n$ y la solución es la misma.

## Definiciones

Durante este informe se utilizarán expresiones/conceptos que pueden causar duda. A continuación se muestran algunos de ellos y su definición.
+ *Solución del problema*: se refiere a un conjunto $S$ de pares $<P_i,K_i,L_i>$ que corresponden a una serie de miradas hechas por Tito. En caso que se obtenga una cantidad de preguntas vistas máxima se dice que es *solución máxima*.
+ *Respuesta del problema*: a diferencia de la solución este es el valor correspondiente a la cantidad máxima de preguntas que se pueden resolver y que responde a lo que se pide en el problema.
+ *Listas o máscaras $\alpha$ y $\beta$*: estas listas son la transformación de las listas $A$ y $B$ respectivamente en una lista de booleanos donde en la posición $i$-ésima se guarda `True` si el entero $i+1$ se encuentra en la lista correspondiente.
+ *Lista $\theta$*: esta lista es resultado de realizar la operación `OR` entre las listas $\alpha$ y $\beta$, es decir, almacena si un entero $i$ se encuentra en alguna de las dos listas.

In [99]:
# El código de esta celda corresponde a la definiciones en Python con las que trabajarán otras celdas de código

from dataclasses import dataclass
from typing import List, Tuple


@dataclass(order=True,repr=True)
class ProblemInput:
    n:int
    k:int
    p:int
    A:List[int]
    B:List[int]

    # Función que construye las listas alpha, beta y theta. CT: O(n)
    def build_masks(self)->None:
        # CT:O(2n)
        self.alpha=[False for _ in range(self.n)]
        self.beta=[False for _ in range(self.n)]

        for x in self.A: # CT:O(n)
            self.alpha[x-1]=True

        for x in self.B: # CT:O(n)
            self.beta[x-1]=True

        # CT:O(n)
        self.theta=[in_a or in_b for in_a,in_b in zip(self.alpha,self.beta)]

    # Función para construir las listas de suma acumulada sobre alpha, beta y theta
    # que permiten calcular la cantidad de preguntas en un intervalo en una sola
    # operación. CT: O(n)
    def build_acc(self) -> None:
        # CT:O(n)
        self.acc_alpha=[0]*(self.n+1)
        self.acc_beta=[0]*(self.n+1)
        self.acc_theta=[0]*(self.n+1)

        # CT:O(n)
        for i in range(self.n):
            self.acc_alpha[i+1]=self.acc_alpha[i]+(1 if self.alpha[i] else 0)
            self.acc_beta[i+1]=self.acc_beta[i]+(1 if self.beta[i] else 0)
            self.acc_theta[i+1]=self.acc_theta[i]+(1 if self.theta[i] else 0)

    # Función para fijar el intervalo en n
    def fix_span(self,left:int,right:int)->Tuple[int,int]:
        a=min(left,right)
        b=max(left,right)
        a=max(a,0)
        b=max(b,0)
        a=min(a,self.n)
        b=min(b,self.n-1)

        return (a,b)

    # Funciones para obtener la cantidad de números presentes en una lista desde left hasta right
    # incluyendo ambos. CT: O(1)
    def count_a(self,left:int, right:int):
        '''Cuenta la cantidad de preguntas del examen que hay en A desde `left` hasta `right` incluyendo ambos'''
        a, b = self.fix_span(left, right)
        l=self.acc_alpha
        return l[b+1]-l[a]

    def count_b(self,left:int, right:int):
        '''Cuenta la cantidad de preguntas del examen que hay en B desde `left` hasta `right` incluyendo ambos'''
        a, b = self.fix_span(left, right)
        l = self.acc_beta
        return l[b+1]-l[a]

    def count_aob(self, left: int, right: int):
        '''Cuenta la cantidad de preguntas del examen que hay en A o en B desde `left` hasta `right` incluyendo ambos'''
        a, b = self.fix_span(left, right)
        l = self.acc_theta
        return l[b+1]-l[a]

    def count_anb(self, left: int, right: int):
        '''Cuenta la cantidad de preguntas del examen que hay en A y no en B desde `left` hasta `right` incluyendo ambos'''
        a, b = self.fix_span(left, right)
        return self.count_aob(left,right)-self.count_b(left,right)

    def count_bna(self, left: int, right: int):
        '''Cuenta la cantidad de preguntas del examen que hay en B y no en A desde `left` hasta `right` incluyendo ambos'''
        a, b = self.fix_span(left, right)
        return self.count_aob(left, right)-self.count_a(left, right)



## Fuerza Bruta Inicial

### Idea
Inicialmente se propone una solución por fuerza bruta cuya idea es recorrer todas las soluciones y comprobar cuál da respuesta a más preguntas. Para recorrer todas las soluciones se recorre cada posibilidad de realizar una cantidad menor que $p$ de miradas en una lista y el resto en la otra y se construye entonces un set con todos los números que se vieron en cada mirada.

### Código

In [100]:
from typing import Iterable

# Un generador de cada lista de len(l) enteros.
# Usado para obtener cada combinación de len(l) posiciones en una lista.
# Lleva en i la posición que debe escribir en la lista l en una ejecución.
def all_list_subspace(n:int,l:List[int],i:int=0)->Iterable[List[int]]:
    # Mira cero veces
    if len(l)==0:
        yield l
    else:
        # Pasa por todas las posiciones a escoger en este punto que sean
        # mayor que las posiciones anteriormente escogidas.
        for k in range(l[i-1]+1 if i!=0 else 0,n):
            # Establece la posición k como la posición de una de las miradas
            l[i]=k
            if i==len(l)-1:
                # Si llegó al final de la lista retórnala
                yield l
            else:
                # Si no está al final tiene varias combinaciones por delante
                # Pasa por cada una de ellas y la retorna
                for y in all_list_subspace(n,l,i+1):
                    yield y


def brute_force(input:ProblemInput):
    input.build_masks()

    answer=0

    for i in range(input.p+1): # Stores how many looks are in list A so the looks in B are p - i
        # Check every possible way to look i positions in a list of n elements
        for choices_a in all_list_subspace(input.n, [0]*i):
            # Check every possible way to look p - i positions in a list of n elements
            for choices_b in all_list_subspace(input.n,[0]*(input.p-i)):
                # Here is analyzed every solution
                # In every look always takes the maximum k possible and different positions
                taken={
                    q for pos in choices_a # take every problem
                    for q in range(pos,min(pos+input.k,input.n)) # that is in the next k position after the one looked
                    if input.alpha[q]} # If the problem is in A

                # Análogo para B. Entonces se toma la union de ambos conjuntos
                taken=taken.union({
                    q for pos in choices_b 
                    for q in range(pos, min(pos + input.k, input.n)) 
                    if input.beta[q]}
                    )

                # Check how many questions get
                answer=max(answer,len(taken))

    return answer

# Basic test cases
# Case 1:
# k=1, p=2
# 12
# 34
case_1=ProblemInput(4,1,2,[1,2],[3,4])
assert brute_force(case_1) == 2, "Fallo en caso 1"

# Case 2:
# k=3, p=2
# 136
# 456
case_2=ProblemInput(6,3,2,[1,3,6],[4,5,6])
assert brute_force(case_2) == 5, "Fallo en caso 2"

# Case 3:
# k=3, p=3
# 1345
# 2678
case_3=ProblemInput(8,3,3,[1,3,4,5],[2,6,7,8])
assert brute_force(case_3) == 7, "Fallo en caso 3"

# Case 4:
# k=1, p=14
# 135
# 2467
case_4=ProblemInput(7,1,14,[1,3,5],[2,4,6,7])
assert brute_force(case_4) == 7, "Fallo en caso 4"

### Complejidad temporal
El algoritmo inicialmente construye las máscaras pasando por cada elemento de cada lista una sola vez (Operación $O(n)$), luego recorre cada solución del problema posible donde en cada mirada se mire el máximo posible, lo cual se traduce en las distintas formas de coger p posiciones a través de las 2 listas. Por combinatoria de conoce que este valor es $C^p_{2n}=\frac{(2n)!}{p!(2n-p)!}=O(2n!)$

Una vez obtenida cada solución posible pasa por cada posición extraída de cada lista, construye un set con todos los números desde cada posición $k$ posiciones en adelante o final del array en caso de tener una posición posterior a $n-k$ (Operación $O(pk)$ entre ambas listas). Luego une ambos sets (correspondiente a los tomados en A y los tomados en B) se unen (Operación $O(pk)$).

Finalmente $T(n,p,k)=O(n)+O(2n!)(O(pk)+O(pk))=O(2n!pk)$

### Correctitud
El algoritmo divide las posibles soluciones según la cantidad de miradas que se hacen en cada lista a través del primer ciclo. Luego conociendo cada lista cuantas miradas tiene se utiliza la función `all_list_subspace` para recorrer una única vez cada combinación de miradas en una lista.

Esta función construye asigna en una lista creada valores correspondientes a cada posición mirada, ya que en cada posición que llena de la lista empieza a asignar valores a partir del anterior asignado se cumple que los enteros están ordenados hasta ese punto, de modo que al llegar al último valor de la lista esta está completamente ordenada. De esta forma, se garantiza que se recorre cada posible selección de miradas en una lista una sola vez puesto que solo existe una forma posible en la que todos los enteros correspondientes a las posiciones de las miradas realizadas (los cuales son distintos ya que no tiene sentido mientras se pueda mirar dos veces una misma posición), estén ordenados.

Luego de tener un conjunto de posiciones miradas en $A$ y en $B$ se encuentra una solución de este problema de forma simplificada al basarse en la siguiente propiedad:

*Para toda solución máxima del problema existe una solución máxima donde en cada posición se mira el máximo de preguntas posibles.*

**Demostración**

Sea $S$ el conjunto de pares de una solución, entonces, para todo $<P_i,K_i,L_i>\in S$ podemos definir un $<P'_i,max(k,n-P_i)>\in S'$ de forma que en $S'$ se ve una cantidad mayor o igual de preguntas que en $S$ (puesto que estamos viendo más posiciones), por tanto, al ser $S$ solución máxima $S'$ también lo es.

Si consideramos las selecciones de posiciones realizadas por el algoritmo como solución con $K_i$ máximo, estamos recorriendo todo el espacio de soluciones de este tipo, luego al encontrar los números que se ven en cada una y guardar la máxima cantidad en `answer` se llega a tener el valor máximo en algún momento y por tanto la respuesta del problema.

## Generador de casos de prueba

In [101]:
import random


def generate_case(min_n, max_n, seed=None):
    r=random.Random(seed)

    # Cantidad aleatoria de preguntas en el examen
    n = r.randint(min_n,max_n)
    
    # Por cada pregunta una probabilidad de un 50% de haberla hecho
    a = [i+1 for i, v in enumerate(r.choices([False, True], k=n)) if v]
    b = [i+1 for i, v in enumerate(r.choices([False, True], k=n)) if v]
    
    # Cantidad de miradas y cantidad de preguntas vistas también aleatoria en un rango
    # que depende de n. Tanto k como p es más probable que sean un número pequeño con respecto a n
    p = int((r.random()**6)*2*n)+1
    k = int((r.random()**4)*n)+1

    return ProblemInput(n,k,p,a,b)

## Tester

In [102]:
from typing import Callable, Tuple


def test(case:ProblemInput,sol:Callable[[ProblemInput],int]):
    return brute_force(case)==sol(case)

def random_test(sol:Callable[[ProblemInput],int],t:int,seed:int=6):
    r=random.Random(seed)
    failed:List[Tuple[int,ProblemInput]]=[]
    ok=0
    for i in range(t):
        if i==0:
            print("Comenzando a probar con el primer caso...\n")
        if (i+1)%10==0:
            failed_text = '\n\t\t'.join([f'Caso {str(i)}: {repr(inp)}' for (i,inp) in failed])
            print(
                "Comenzando caso "+f"{i+1}"+"...\nResultados hasta ahora:\n\tOK/FALLADOS/TOTAL:"+
                f"{ok}/{len(failed)}/{i}"+("\n\tFALLADOS:"+f"{failed_text}" if len(failed)!=0 else "")+"\n")
        p_in=generate_case(1,12,seed)

        # Cambia la semilla para el generador de cada caso en función de la semilla original
        seed=r.random()

        # Salió bien
        if test(p_in,sol):
            ok+=1
        else:
            failed.append((i+1,p_in))
    print(
        "Test finalizado" +
        "\nResultados:\n\tOK/FALLADOS/TOTAL:" +
        f"{ok}/{len(failed)}/{t}" +
        ("\n\tFALLADOS:" + f"{failed_text}" if len(failed) != 0 else "") + "\n\n")

# Solo para evaluar el funcionamiento del tester
random_test(brute_force,25)

Comenzando a probar con el primer caso...

Comenzando caso 10...
Resultados hasta ahora:
	OK/FALLADOS/TOTAL:9/0/9

Comenzando caso 20...
Resultados hasta ahora:
	OK/FALLADOS/TOTAL:19/0/19

Test finalizado
Resultados:
	OK/FALLADOS/TOTAL:25/0/25




## Intentos de solución
Los intentos de encontrar una solución mejor a partir de patrones fueron muchos. 

Inicialmente se pensó que se podía hallar una respuesta para el problema dado un $p$ si se conocía una solución para $p-1$. La dificultad no vista aquí es que existen soluciones $p=a$ que a partir de otra acción no se puede obtener una solución máxima para $p=k+1$. Un ejemplo puede ser el siguiente caso:

$$ n=6, k=3, p=1$$
$$ A=[1,2,3,4,5,6], B=[]$$

Una solución posible es $S={<2,3,A>}$ con resultado 3 donde se mira desde la segunda pregunta hasta la $4$, pero, si se analiza a partir de ahí, tomar otra decisión solo puede aumentar la cantidad de ejercicios vista en a $2$ máximo dando como resultado $5$, mientras que para $p=2$ la respuesta es en realidad $6$. Por otro lado la también solución $S'={<1,3,A>}$ si permite esta estrategia. Esto hace pensar en la posibilidad de que siempre exista una solución que cumpla con la propiedad de que al seguir esta estrategia se obtenga una solución máxima, y que además esta podría ser alguna de las que más temprano toma posiciones; pero esta hipótesis no pudo ser demostrada.

Analizando un ejemplo como el anterior se consideró también analizar intervalos de tamaño $k$ y la cantidad de preguntas en ese intervalo en cada lista. De forma intuitiva los intervalos con mayor cantidad de preguntas serían mejores para escoger. Pero hay dos listas y tomar un intervalo bueno en una puede convertir a un intervalo en malo en la otra. Esta idea apoyó la solución final pero en esta línea no se encontró formalización ni patrón que permitiera hacer seguimiento de este proceso y acercarse a una solución greedy.

Ya analizada una posibilidad de una solución con programación dinámica y greedy se fuerza una con búsqueda binaria a través de la siguiente idea: si fijamos un valor para la respuesta $r$ y modelamos el problema como *encontrar si existe una solución tal que la respuesta del problema sea $r$* entonces se cumple que si $r$ es valor máximo no existe solución con mejor respuesta y su vez existe solución con peor resultado, por tanto se puede hacer una búsqueda binaria de este valor máximo. Esta línea no se continuó ya que el problema de encontrar si existe una solución con tal respuesta se percibió igual de complejo que el problema original.

Volviendo a la idea con programación dinámica se pensó entonces llevar una mejor solución $S_i$ o su respuesta $R_i$ de forma que sea la mejor posible considerando hasta la $i$-ésima pregunta o hasta una posición determinada en las listas $A$ y $B$. La dificultad en un principio para esta idea era formalizar como se comportaría en un caso tal que para una posición $j$, $R_i$ con $i>j$ dependiera de haber mirado en una posición $l<j$; la idea es que a medida que se descubren preguntas de la prueba la mejor decisión en las últimas posiciones puede cambiar.

Para solucionar esto se pudiera analizar en una posición $j$ las mejores soluciones desde $j-k$ hasta $j$ sin mirar en ninguna posición y mirando en cada una de las posiciones mencionadas. Pero en esa idea hay un error, y es que al mirar hacia atrás es difícil llevar cuanto se ha cogido de una lista y de otra, es decir, las intersecciones entre los intervalos tomados de las dos listas son complicadas de solucionar. De alguna forma se tendría que tener en cuenta estas posibilidades a la hora de calcular la respuesta y es entonces donde entra la solución propuesta.

## Solución Propuesta

### Idea General
Teniendo en cuenta las ideas anteriores debemos analizar las mejores soluciones a partir de: una posición en la que estamos y quizás tomemos, hasta donde puede haber cogido la otra lista y además la lista que vamos analizando y las miradas que hemos hecho. Esos datos constituyen el estado de la solución propuesta por programación dinámica:
+ $i$: la posición actualmente analizada.
+ $k'$: cuan adelantada está la otra lista con respecto a la actualmente analizada.
+ $l$: cuántas miradas se han hecho.
Cada uno de estos se considera de manera independiente por cada lista en dos tablas donde el valor almacenado en cada tabla asume que la última mirada se hizo en la lista que le corresponde.

Inicialmente todos los valores serán cero y la idea es intentar tomar posiciones de forma inteligente y ver si mejora la cantidad de preguntas obtenidas. Para solucionar el problema de saber si las preguntas a tomar ya fueron tomadas se considera cuanto podría estar tomado por delante la otra lista en cada posición. Al pasar por cada estado se analiza la posibilidad de no hacer ninguna mirada, mirar en $A$, mirar en b o mirar en las $2$ (en caso de que sea un estado en el que ambas listas estén parejas).

La idea es que al llevar el registro de cuan adelantada se encuentra una lista en revisión por encima de otra se pueda, al decidir realizar otra mirada, definir exactamente los números que ya se han tomado y los que no para realizar el cálculo correctamente.

### Código

In [103]:
import numpy as np

def solution(p_in:ProblemInput)->int:
    p_in.build_masks()
    p_in.build_acc()

    n=p_in.n
    k=p_in.k
    p=p_in.p

    # dpa[i][k'][l] indica que hemos analizado hasta la posición i en los dos,
    # que en b hemos tomado k' posiciones de más en la ultima mirada 
    # (es decir, ya están tomadas las posiciones i+1, i+2, ..., i+k') y hemos 
    # hecho en total l miradas
    dpa = np.zeros((n+1, k, p+1), dtype=int)
    dpb = np.zeros((n+1, k, p+1), dtype=int)
    
    for i in range(p_in.n):
        for kp in range(p_in.k):
            for l in range(p_in.p):
                if kp==0: # las listas van por la misma posición

                    #
                    # Transiciones de dpa
                    #

                    # No se toma/mira nada en ningún lado
                    # Si están parejo y no se avanza nada lo mejor hasta el momento en esta lista(a) 
                    # se compara con lo mejor que se tiene en la posición siguiente en ambas listas
                    dpa[i+1,0,l] = max(dpa[i+1,0,l], dpa[i+1,0,l])
                    dpb[i+1,0,l] = max(dpb[i+1,0,l], dpa[i+1,0,l])

                    # Toma un intervalo en a
                    dpb[i + 1][k - 1][l + 1] = max(
                        dpb[i + 1][k - 1][l + 1],
                        dpa[i][0][l] + p_in.count_a(i, i + k - 1))
                    # Tomar un intervalo en b
                    dpa[i + 1][k - 1][l + 1] = max(
                        dpa[i + 1][k - 1][l + 1],
                        dpa[i][0][l] + p_in.count_b(i, i + k - 1))
                    # Tomar un intervalo en los dos
                    if (l + 2 <= p):
                        dpa[min(n, i + k)][0][l + 2] = max(
                            dpa[min(n, i + k)][0][l + 2],
                             dpa[i][0][l] + p_in.count_aob(i, i + k - 1))
                        dpb[min(n, i + k)][0][l + 2] = max(
                            dpb[min(n, i + k)][0][l + 2], 
                            dpa[i][0][l] + p_in.count_aob(i, i + k - 1))

                    #
                    # Transiciones de dpb
                    #

                    dpa[i + 1][0][l] = max(dpa[i + 1][0][l], dpb[i][0][l])
                    dpb[i + 1][0][l] = max(dpb[i + 1][0][l], dpb[i][0][l])
                    dpb[i + 1][k - 1][l + 1] = max(
                        dpb[i + 1][k - 1][l + 1], 
                        dpb[i][0][l] + p_in.count_a(i, i + k - 1))
                    dpa[i + 1][k - 1][l + 1] = max(
                        dpa[i + 1][k - 1][l + 1], 
                        dpb[i][0][l] + p_in.count_b(i, i + k - 1))
                    if (l + 2 <= p):
                        dpa[min(n, i + k)][0][l + 2] = max(
                            dpa[min(n, i + k)][0][l + 2], 
                            dpb[i][0][l] + p_in.count_aob(i, i + k - 1))
                        dpb[min(n, i + k)][0][l + 2] = max(
                            dpb[min(n, i + k)][0][l + 2], 
                            dpb[i][0][l] + p_in.count_aob(i, i + k - 1))
                else: # Una de las listas va por delante

                    #
                    # Transiciones en a
                    #

                    # no tomar nada en a
                    dpa[i + 1][kp - 1][l] = max(dpa[i + 1][kp - 1][l], dpa[i][kp][l])
                    if (kp - 1 == 0):
                        dpb[i + 1][0][l] = max(dpb[i + 1][0][l], dpa[i][kp][l])
                    # tomar un intervalo en a
                    dpb[min(n, i + kp)][k - kp][l + 1] = max(
                        dpb[min(n, i + kp)][k - kp][l + 1], 
                        dpa[i][kp][l] + p_in.count_anb(i, i + kp - 1) + p_in.count_a(i + kp, i + k - 1))

                    #
                    # Transiciones en b
                    #

                    # no tomar nada en b
                    dpb[i + 1][kp - 1][l] = max(dpb[i + 1][kp - 1][l], dpb[i][kp][l])
                    if (kp - 1 == 0):
                        dpa[i + 1][0][l] = max(dpa[i + 1][0][l], dpb[i][kp][l])
                    # tomar un intervalo en b
                    dpa[min(n, i + kp)][k - kp][l + 1] = max(
                        dpa[min(n, i + kp)][k - kp][l + 1], 
                        dpb[i][kp][l] + p_in.count_bna(i, i + kp - 1) + p_in.count_b(i + kp, i + k - 1))

    result = 0
    for i in range(p_in.n+1):
        for kp in range(p_in.k):
            for l in range(1,p_in.p+1):
                result = max(result, dpa[i][kp][l])
                result = max(result, dpb[i][kp][l])
    
    return result

In [104]:
random_test(solution,30)

Comenzando a probar con el primer caso...

Comenzando caso 10...
Resultados hasta ahora:
	OK/FALLADOS/TOTAL:9/0/9

Comenzando caso 20...
Resultados hasta ahora:
	OK/FALLADOS/TOTAL:19/0/19

Comenzando caso 30...
Resultados hasta ahora:
	OK/FALLADOS/TOTAL:29/0/29

Test finalizado
Resultados:
	OK/FALLADOS/TOTAL:30/0/30




### Complejidad temporal
Inicialmente el algoritmo realiza un precálculo para obtener las listas $\alpha$, $\beta$ y $\theta$ y otro para hallar la lista de sumas acumulativas de estas para posibilitar la cantidad de números de un intervalo en tiempo constante. Esta operación es lineal por cada lista construida lo que la hace $O(n)$.

Posteriormente se crean las matrices correspondientes a las tablas de ambas dinámicas, la cantidad de operaciones es a lo sumo la cantidad de elementos de ambas por una constante, lo cual lo hace $O(npk)$.

Luego de esto se realizan los ciclos principales del algoritmo, los cuales realiza operaciones $O(1)$ en su interior y recorren cada elemento de las matrices, por tanto es $O(npk)$. Para dar la respuesta final se recorren todos los estados/elementos de ambas matrices para encontrar el máximo, esto es $O(npk)$.

Complejidad final: $O(n)+O(npk)+O(npk)O(1)+O(npk)=O(npk)$